# Semantic Kernel Tool calling capabilities with Unity Catalog

## Prerequisites

**API Key**
To run this tutorial, you will need an OpenAI API key. 

Once you have acquired your key, set it to the environment variable `OPENAI_API_KEY`.

Below, we validate that this key is set properly in your environment.

**Packages**

To interface with both UnityCatalog and Semantic Kernel, you will need to install the following package:

```shell
pip install unitycatalog-semantic-kernel
```

**Note**
This example uses Semantic Kernel with Unity Catalog to create and use functions as tools within your AI applications.

In [1]:
# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os

assert "OPENAI_API_KEY" in os.environ, (
    "Please set the OPENAI_API_KEY environment variable to your OpenAI API key"
)

## Configuration and Client setup

In order to connect to your Unity Catalog server, you'll need an instance of the `DatabricksFunctionClient` from the `unitycatalog-client` package. 

> Note: If you don't already have a Catalog and a Schema created, be sure to create them before running this notebook and adjust the `CATALOG` and `SCHEMA` variables below to suit.

In [3]:
from unitycatalog.ai.core.databricks import DatabricksFunctionClient
from unitycatalog.ai.semantic_kernel.toolkit import UCFunctionToolkit
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.prompt_execution_settings import PromptExecutionSettings
from semantic_kernel.contents.chat_history import ChatHistory

In [4]:
# Initialize the Databricks client
client = DatabricksFunctionClient(profile='e2-dogfood')

CATALOG = "AICatalog"
SCHEMA = "AISchema"

## Define a function and register it to Unity Catalog

In this next section, we'll be defining a Python function and creating it within Unity Catalog so that it can be retrieved and used as a tool by Semantic Kernel. 

There are a few things to keep in mind when creating functions for use with the `create_python_function` API:

- Ensure that your have properly defined types for all arguments and for the return of the function.
- Ensure that you have a Google-style docstring defined that includes descriptions for the function, each argument, and the return of the function. This is critical, as these are used to populate the metadata associated with the function within Unity Catalog, providing contextual data for an LLM to understand when and how to call the tool associated with this function.
- If there are packages being called that are not part of core Python, ensure that the import statements are locally scoped (defined within the function body).

In [5]:
def add_numbers(a: float, b: float) -> float:
    """
    Adds two numbers and returns the result.

    Args:
        a (float): First number.
        b (float): Second number.

    Returns:
        float: The sum of the two numbers.
    """
    return a + b

In [6]:
# Create the function within Unity Catalog
function_info = client.create_python_function(
    func=add_numbers,
    catalog=CATALOG,
    schema=SCHEMA,
    replace=True  # Set to True to overwrite if the function already exists
)

print("Created function:", function_info)

Created function: FunctionInfo(browse_only=None, catalog_name='puneetjain_uc', comment='Adds two numbers and returns the result.', created_at=1743154743012, created_by='puneet.jain@databricks.com', data_type=<ColumnTypeName.DOUBLE: 'DOUBLE'>, external_language='Python', external_name=None, full_data_type='DOUBLE', full_name='puneetjain_uc.autogen_ucai.add_numbers', function_id='df2d6256-8a20-482a-b231-530b4b7a26b5', input_params=FunctionParameterInfos(parameters=[FunctionParameterInfo(name='a', type_text='double', type_name=<ColumnTypeName.DOUBLE: 'DOUBLE'>, position=0, comment='First number.', parameter_default=None, parameter_mode=None, parameter_type=<FunctionParameterType.PARAM: 'PARAM'>, type_interval_type=None, type_json='{"name":"a","type":"double","nullable":true,"metadata":{"comment":"First number."}}', type_precision=0, type_scale=0), FunctionParameterInfo(name='b', type_text='double', type_name=<ColumnTypeName.DOUBLE: 'DOUBLE'>, position=1, comment='Second number.', paramete

## Set up Semantic Kernel

Now we'll set up Semantic Kernel with the necessary components for using our Unity Catalog function as a tool.

In [8]:
# Initialize the kernel with an AI service
kernel = Kernel()

# Add chat completion service
chat_completion_service = OpenAIChatCompletion(
    ai_model_id='gpt-4',
    api_key=os.getenv("OPENAI_API_KEY")
)

# Set up execution settings
settings = PromptExecutionSettings(
    function_choice_behavior=FunctionChoiceBehavior.Auto(),
)

## Create a Toolkit instance of the function

Now that the function has been created within Unity Catalog, we can use the `unitycatalog-semantic-kernel` package to create a toolkit instance that Semantic Kernel will use to access our function.

In [9]:
# Create toolkit instance
toolkit = UCFunctionToolkit(
    function_names=[f"{CATALOG}.{SCHEMA}.add_numbers"],
    client=client
)

# Register Unity Catalog functions with the kernel
toolkit.register_with_kernel(kernel, plugin_name="calculator")

Kernel(retry_mechanism=PassThroughWithoutRetry(), services={}, ai_service_selector=<semantic_kernel.services.ai_service_selector.AIServiceSelector object at 0x173842c50>, plugins={'calculator': KernelPlugin(name='calculator', description=None, functions={'add_numbers': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='add_numbers', plugin_name='calculator', description='Adds two numbers and returns the result.', parameters=[KernelParameterMetadata(name='a', description='First number.', default_value=None, type_='double', is_required=True, type_object=<class 'float'>, schema_data={'type': 'number', 'description': 'First number.'}, include_in_function_choices=True), KernelParameterMetadata(name='b', description='Second number.', default_value=None, type_='double', is_required=True, type_object=<class 'float'>, schema_data={'type': 'number', 'description': 'Second number.'}, include_in_function_choices=True)], is_prompt=False, is_asynchronous=False, return_parameter=KernelPar

## Create and Process Chat History

Now we'll create a chat history and process it using our Semantic Kernel setup.

In [10]:
# Create chat history
chat_history = ChatHistory()
chat_history.add_user_message(
    """You are a helpful calculator assistant. Use the calculator tools to answer questions about numbers.
    Question: What is 49 + 82?"""
)

In [11]:
# Process the chat interaction
response = await chat_completion_service.get_chat_message_content(
    chat_history,
    settings,
    kernel=kernel
)

print("\nFinal Response:", response)


Final Response: The result of 49 + 82 is 131.


## View Chat History

Let's look at the complete chat history to see how the interaction unfolded.

In [12]:
print("\nChat History:")
print("-" * 80)
for message in chat_history.messages:
    print(f"Role: {message.role}")
    print(f"Content: {message.content}")
    if message.content == "":
        print(f"Details: {message.items}")
    print("-" * 80)


Chat History:
--------------------------------------------------------------------------------
Role: AuthorRole.USER
Content: You are a helpful calculator assistant. Use the calculator tools to answer questions about numbers.
    Question: What is 49 + 82?
--------------------------------------------------------------------------------
Role: AuthorRole.ASSISTANT
Content: 
Details: [FunctionCallContent(inner_content=None, ai_model_id=None, metadata={}, content_type='function_call', id='call_vYGDqditbrOVBXiLIKIQDi25', index=None, name='calculator-add_numbers', function_name='add_numbers', plugin_name='calculator', arguments='{\n  "a": 49,\n  "b": 82\n}')]
--------------------------------------------------------------------------------
Role: AuthorRole.TOOL
Content: 
Details: [FunctionResultContent(inner_content=FunctionResult(function=KernelFunctionMetadata(name='add_numbers', plugin_name='calculator', description='Adds two numbers and returns the result.', parameters=[KernelParameterMe